<a href="https://colab.research.google.com/github/raheeltahir55/CE888/blob/main/Project/Proj_MultiEval_Emotion_RoBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 20.7MB/s 
     |████████████████████████████████| 3.3MB 52.4MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 


In [2]:
### EMOTION####
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import io
import requests

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
pip install nlpaug

     |████████████████████████████████| 399kB 19.3MB/s 


In [6]:
pip install torch>=1.6.0 transformers>=4.0.0

In [7]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

In [8]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt', allow_redirects= True)
open('test_text.txt', 'wb').write(r.content)
example1 = "/content/test_text.txt"
with open(example1, "r") as file1:
    data = file1.readlines()

In [9]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt', allow_redirects= True)
open('test_labels.txt', 'wb').write(r.content)
example1 = "/content/test_labels.txt"
with open(example1, "r") as file1:
    data1 = file1.readlines()

In [10]:
data= [data[i].strip() for i in range(len(data))]
data1= [data1[i].strip() for i in range(len(data1))]

In [11]:
true_label= np.array(data1).astype(np.int32)
print(true_label)

[3 0 3 ... 0 0 1]


In [12]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [13]:
data=[preprocess(data[i]) for i in range(len(data))]

In [14]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")

In [15]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
# MODEL = f"cardiffnlp/bertweet-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [16]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [17]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
pred_label= []

for text in data:
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  pred= np.argmax(scores)
  pred_label.append(pred)

In [18]:
pred_label= np.array(pred_label)

In [19]:
print("F1 Score -> ",f1_score(pred_label, true_label, average='macro')*100)

F1 Score ->  79.8272412305532


In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good work hi how are you? hello mate?"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

In [ ]:
separator=' '
text =['good','work','hi', 'how', 'are', 'you']
separator.join(text)

'good work hi how are you'

In [ ]:
print(encoded_input)

{'input_ids': tensor([[    0, 29042,    28,   655,    98,  6161,   114,    47,   115,   638,
          6081,     9,   286,   571, 12367, 22164,   787, 12105,   787, 12105,
           849,   424, 22736,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print(scores)

[0.00730037 0.04755703 0.9451425 ]


In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")